# KNOWN FACES INCREMENT

This code is used to increment "faces" database table with known faces

In [2]:
import warnings
warnings.filterwarnings("ignore")

from functions import *

import numpy as np
import cv2

### INCREMENT DATABASE WITH HISTORY

In [5]:
image = cv2.imread("src/angry.jpeg")
features = extract_features(image)

create_embeddings("Angry", features)


Face with name Angry already exists



I0000 00:00:1723899313.207008 15731065 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1723899313.209424 15731962 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723899313.212582 15731963 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
